In [1]:
# Load our modules
import numpy as np 
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as matplot

import re
import sklearn

# Ignore any warnings for cleaner output (not necessary)
import warnings
warnings.filterwarnings("ignore")

In [3]:
df1  = pd.read_csv("https://raw.githubusercontent.com/rajeevratan84/data-analyst-bootcamp/master/labs.csv")
df2  = pd.read_csv("https://raw.githubusercontent.com/rajeevratan84/data-analyst-bootcamp/master/examination.csv")
df3  = pd.read_csv("https://raw.githubusercontent.com/rajeevratan84/data-analyst-bootcamp/master/demographic.csv")
df4  = pd.read_csv("https://raw.githubusercontent.com/rajeevratan84/data-analyst-bootcamp/master/diet.csv")
df5  = pd.read_csv("https://raw.githubusercontent.com/rajeevratan84/data-analyst-bootcamp/master/questionnaire.csv")

df2.drop(['SEQN'], axis = 1, inplace=True)
df3.drop(['SEQN'], axis = 1, inplace=True)
df4.drop(['SEQN'], axis = 1, inplace=True)
df5.drop(['SEQN'], axis = 1, inplace=True)

df = pd.concat([df1, df2], axis=1, join='inner')
df = pd.concat([df, df3], axis=1, join='inner')
df = pd.concat([df, df4], axis=1, join='inner')
df = pd.concat([df, df5], axis=1, join='inner')

#sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
#sel.fit_transform(df)

df.describe()

SEQN       URXUMA  ...       WHQ500       WHQ520
count   9813.000000  8052.000000  ...  1424.000000  1424.000000
mean   78644.559971    41.218854  ...     2.295646     1.747893
std     2938.592266   238.910226  ...     1.210905     0.707600
min    73557.000000     0.210000  ...     1.000000     1.000000
25%    76092.000000     4.500000  ...     1.000000     1.000000
50%    78643.000000     8.400000  ...     2.000000     2.000000
75%    81191.000000    17.625000  ...     3.000000     2.000000
max    83731.000000  9600.000000  ...     9.000000     9.000000

[8 rows x 1781 columns]

In [4]:
df.head()

SEQN  URXUMA  URXUMS  URXUCR.x  ...  WHQ150  WHQ030M  WHQ500  WHQ520
0  73557     4.3     4.3      39.0  ...    62.0      NaN     NaN     NaN
1  73558   153.0   153.0      50.0  ...    25.0      NaN     NaN     NaN
2  73559    11.9    11.9     113.0  ...    35.0      NaN     NaN     NaN
3  73560    16.0    16.0      76.0  ...     NaN      3.0     3.0     3.0
4  73561   255.0   255.0     147.0  ...    60.0      NaN     NaN     NaN

[5 rows x 1812 columns]

In [5]:
df.dropna(axis=1, how='all')
df.dropna(axis=0, how='all')

df = df.rename(columns = {'SEQN' : 'ID',
                          'RIAGENDR' : 'Gender',
                          'DMDYRSUS' : 'Years_in_US', # Nan -> american i guess
                          'INDFMPIR' : 'Family_income',
                          'LBXGH' : 'GlycoHemoglobin',
                          'BMXARMC' : 'ArmCircum',
                          'BMDAVSAD' : 'SaggitalAbdominal',
                          'MGDCGSZ' : 'GripStrength',
                          'DRABF' : 'Breast_fed'})

df = df.loc[:, ['ID', 'Gender', 'Years_in_US', 'Family_income','GlycoHemoglobin', 'ArmCircum', 
                'SaggitalAbdominal', 'GripStrength', 'Breast_fed']]

df.describe()

ID       Gender  ...  GripStrength   Breast_fed
count   9813.000000  9813.000000  ...   7677.000000  8661.000000
mean   78644.559971     1.509426  ...     63.054891     1.984528
std     2938.592266     0.499937  ...     25.098439     0.123426
min    73557.000000     1.000000  ...      8.000000     1.000000
25%    76092.000000     1.000000  ...     45.600000     2.000000
50%    78643.000000     2.000000  ...     60.300000     2.000000
75%    81191.000000     2.000000  ...     80.700000     2.000000
max    83731.000000     2.000000  ...    162.800000     2.000000

[8 rows x 9 columns]

In [6]:
from sklearn.feature_selection import VarianceThreshold

#year in us -> american : 0, not american : 1
df.dropna(axis=1, how='all')
df.dropna(axis=0, how='all')

#YEARS IN US NA처리
df['Years_in_US'] = df['Years_in_US'].apply(lambda x: x if x > 0 else 0)

#GlycoHemoglobin, Saggital Abdominal(median)
df['GlycoHemoglobin'] = df['GlycoHemoglobin'].fillna(df['GlycoHemoglobin'].median())
df['SaggitalAbdominal'] = df['SaggitalAbdominal'].fillna(df['SaggitalAbdominal'].median())
df['ArmCircum'] = df['ArmCircum'].fillna(df['ArmCircum'].median())
df['GripStrength'] = df['GripStrength'].fillna(df['GripStrength'].median())

#Family Income -> use ffill to fill na
df['Family_income'] = df['Family_income'].fillna(method='ffill')

#Breat_fed -> fill to 1
df['Breast_fed'] = df['Breast_fed'].fillna(value = 1)

# We remove all features with low variance which leaves us with the dataframe below
# Features with a training-set variance lower than this threshold will be removed.
sel = VarianceThreshold(threshold=(0.15))
sel.fit_transform(df)

df.describe()

ID       Gender  ...  GripStrength   Breast_fed
count   9813.000000  9813.000000  ...   9813.000000  9813.000000
mean   78644.559971     1.509426  ...     62.455233     1.868949
std     2938.592266     0.499937  ...     22.228196     0.337473
min    73557.000000     1.000000  ...      8.000000     1.000000
25%    76092.000000     1.000000  ...     50.300000     2.000000
50%    78643.000000     2.000000  ...     60.300000     2.000000
75%    81191.000000     2.000000  ...     73.300000     2.000000
max    83731.000000     2.000000  ...    162.800000     2.000000

[8 rows x 9 columns]

In [7]:
df.loc[df['GlycoHemoglobin'] < 6.0, 'Diabetes'] = 0
df.loc[(df['GlycoHemoglobin'] >= 6.0) & (df['GlycoHemoglobin'] <= 6.4), 'Diabetes'] = 1
df.loc[df['GlycoHemoglobin'] >= 6.5, 'Diabetes'] = 2

df.head(10)

ID  Gender  Years_in_US  ...  GripStrength  Breast_fed  Diabetes
0  73557       1          0.0  ...          55.2         2.0       2.0
1  73558       1          0.0  ...          61.5         2.0       2.0
2  73559       1          0.0  ...          91.0         2.0       2.0
3  73560       1          0.0  ...          32.2         2.0       0.0
4  73561       2          0.0  ...          30.9         2.0       0.0
5  73562       1          0.0  ...          53.1         2.0       0.0
6  73563       1          0.0  ...          60.3         1.0       0.0
7  73564       2          0.0  ...          45.9         2.0       0.0
8  73566       1          0.0  ...          38.8         2.0       0.0
9  73567       2          0.0  ...          43.5         2.0       0.0

[10 rows x 10 columns]

In [8]:
colormap = plt.cm.viridis
plt.figure(figsize=(10,10))
sns.heatmap(df.astype(float).drop(axis=1, labels='ID').corr(), linewidths=0.1, vmax=1.0, square=True, cmap=colormap, annot=True)

In [9]:
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score

In [10]:
df.drop(['GlycoHemoglobin'], axis = 1, inplace=True)

df.head(5)

ID  Gender  Years_in_US  ...  GripStrength  Breast_fed  Diabetes
0  73557       1          0.0  ...          55.2         2.0       2.0
1  73558       1          0.0  ...          61.5         2.0       2.0
2  73559       1          0.0  ...          91.0         2.0       2.0
3  73560       1          0.0  ...          32.2         2.0       0.0
4  73561       2          0.0  ...          30.9         2.0       0.0

[5 rows x 9 columns]

In [11]:
from sklearn.linear_model import LinearRegression

#data -> attributes, target -> diabetes
data = df.drop(['Diabetes'], axis=1)
target = df[['Diabetes']]

#seperate training set and test set
train_X = data[:6000]
test_X = data[6000:]
train_Y = target[:6000]
test_Y = target[6000:]

#create linear regression obj
lr_regr = LinearRegression()

#training via linear regression model
lr_regr.fit(train_X, train_Y)

#make prediction using the test set
lr_pred_diabetes = lr_regr.predict(test_X)
lr_score = lr_regr.score(test_X, test_Y)

print('LRr_Coefficients: ', lr_regr.coef_)
print('LR_Mean Square Error: %.2f' % mean_squared_error(test_Y, lr_pred_diabetes))
print('LR_Variance score: %.2f' % r2_score(test_Y, lr_pred_diabetes))
print('Score: %.2f' % lr_regr.score(test_X, test_Y))

LRr_Coefficients:  [[ 1.98953563e-06 -2.46852260e-02 -9.10149905e-04 -5.26378493e-03
   1.12229802e-02  3.57806364e-02 -3.02433944e-03  1.73096160e-03]]
LR_Mean Square Error: 0.23
LR_Variance score: 0.12
Score: 0.12


In [12]:
lr_pred_diabetes

array([[0.17610814],
       [0.06915668],
       [0.1792808 ],
       ...,
       [0.46134257],
       [0.19766727],
       [0.09884146]])

In [13]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

#create adaboost classification obj
ab_clf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100, 
                            learning_rate=0.5, random_state=100)

#training via adaboost classficiation model
ab_clf.fit(train_X, train_Y)
print("training....\n")

#make prediction using the test set
ab_pred_diabetes = ab_clf.predict(test_X)
print('prediction: \n', ab_pred_diabetes)

print('\nparms: \n', ab_clf.get_params)

#score
ab_clf_score = ab_clf.score(test_X, test_Y)
print("\nmean accuracy: %.2f" % ab_clf.score(test_X, test_Y))

training....

prediction: 
 [0. 0. 0. ... 0. 0. 0.]

parms: 
 <bound method BaseEstimator.get_params of AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                         class_weight=None,
                                                         criterion='gini',
                                                         max_depth=None,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0

In [14]:
from sklearn.ensemble import BaggingClassifier

bagging = BaggingClassifier(base_estimator= DecisionTreeClassifier(), max_samples = 0.5, max_features = 0.5, 
                            bootstrap = False, bootstrap_features = False)

bagging.fit(train_X, train_Y)
bg_pred_diabetes = bagging.predict(test_X)

bg_dt_score = bagging.score(test_X, test_Y)
bagging.score(test_X, test_Y)

0.8785733018620508